In [26]:
"""
Travelling Salesman Problem Solver

This script implements a heuristic algorithm for the Travelling Salesman Problem (TSP) that uses the 2-opt local search to optimize an initial solution generated by the Randomized Nearest Neighbor algorithm. The TSP seeks to find the shortest route that visits every city in a given list exactly once and returns to the starting city.

This script requires Python 3.6 or later and depends on the `math` module.

Usage:
    Call the `tsp_solver` function with a list of cities and their coordinates, a distance matrix that stores the distance between every pair of cities, the number of iterations to run the algorithm, and the unit of measurement for the distance traveled. The function returns the best route found and its total distance in the specified unit of measurement.

Example:
    cities = [(0,0), (1,1), (2,2), (3,3), (4,4), (5,5)]
    dist_matrix = [[0, 1, 2, 3, 4, 5],
                   [1, 0, 1, 2, 3, 4],
                   [2, 1, 0, 1, 2, 3],
                   [3, 2, 1, 0, 1, 2],
                   [4, 3, 2, 1, 0, 1],
                   [5, 4, 3, 2, 1, 0]]
    iterations = 1000
    unit = 'kilometers'
    best_route, total_distance = tsp_solver(cities, dist_matrix, iterations, unit)
    print(f"Best route: {best_route}\nTotal distance: {total_distance} {unit}")

++++++++++++++++++++ FUNCTIONS TO BE IMPLEMENTED START ++++++++++++++++++++

Function: tsp_solver

    Solves the TSP for a given list of cities and their coordinates, distance matrix, number of iterations, and unit of
    measurement. Returns the best route and its total distance in the specified unit of measurement.

    Parameters:
        cities (list of tuples): A list of cities with their coordinates.
        dist_matrix (2D list): A distance matrix that stores the distance between every pair of cities.
        iterations (int): The number of iterations to run the algorithm.
        unit (str): The unit of measurement for the distance traveled (e.g. 'kilometers', 'miles', 'meters').

    Returns:
        tuple: A tuple containing the best route found and its total distance in the specified unit of measurement.

Function: randomized_nearest_neighbor

    Generates an initial solution to the TSP using the randomized nearest neighbor algorithm.

    Parameters:
        cities (list of tuples): A list of cities with their coordinates.
        dist_matrix (2D list): A distance matrix that stores the distance between every pair of cities.

                    #Distance Matrix Example
                    City A  City B  City C  City D
            City A       0     10      15      20
            City B      10      0      25      30
            City C      15     25       0      35
            City D      20     30      35       0

    Returns:
        list: A list representing the initial route generated by the algorithm.
    
Function: calculate_total_distance

    Calculates the total distance of a route given its distance matrix.

    Parameters:
        route (list): A list representing a route.
        dist_matrix (2D list): A distance matrix that stores the distance between every pair of cities.

    Returns:
        float: The total distance of the route.
    
Function: swap_cities

    Swaps two cities in a route.

    Parameters:
        route (list): A list representing a route.
        index1 (int): The index of the first city to swap.
        index2 (int): The index of the second city to swap.

    Returns:
        list: A list representing the new route generated by the swap.
    
Function: apply_2opt_swap

    Applies the 2-opt swap to a route.

    Parameters:
        route (list): A list representing a route.
        index1 (int): The index of the first city to swap.
        index2 (int): The index of the second city to swap.

    Returns:
        list: A list representing the new route generated by the 2-opt swap.

Function: tsp_algorithm

    Runs the TSP algorithm for a given number of iterations.

    Parameters:
        route (list): A list representing a route.
        dist_matrix (2D list): A distance matrix that stores the distance between every pair of cities.
        iterations (int): The number of iterations to run the algorithm.

    Returns:
        tuple: A tuple containing the best route found and its total distance.
    
Function: tsp_solver

    Solves the TSP for a given list of cities and their coordinates, distance matrix, number of iterations, and unit of
    measurement. Returns the best route and its total distance in the specified unit of measurement.

    Parameters:
        cities (list of tuples): A list of cities with their coordinates.
        dist_matrix (2D list): A distance matrix that stores the distance between every pair of cities.
        iterations (int): The number of iterations to run the algorithm.
        unit (str): The unit of measurement for the distance traveled (e.g. 'kilometers', 'miles', 'meters').

    Returns:
        tuple: A tuple containing the best route found and its total distance in the specified unit of measurement.

++++++++++++++++++++ FUNCTIONS TO BE IMPLEMENTED END ++++++++++++++++++++

Author: Carl Kho
Date created: March 8, 2023
Last modified: March 8, 2023
Python Version: 3.6 or later
"""

'\nTravelling Salesman Problem Solver\n\nThis script implements a heuristic algorithm for the Travelling Salesman Problem (TSP) that uses the 2-opt local search to optimize an initial solution generated by the Randomized Nearest Neighbor algorithm. The TSP seeks to find the shortest route that visits every city in a given list exactly once and returns to the starting city.\n\nThis script requires Python 3.6 or later and depends on the `math` module.\n\nUsage:\n    Call the `tsp_solver` function with a list of cities and their coordinates, a distance matrix that stores the distance between every pair of cities, the number of iterations to run the algorithm, and the unit of measurement for the distance traveled. The function returns the best route found and its total distance in the specified unit of measurement.\n\nExample:\n    cities = [(0,0), (1,1), (2,2), (3,3), (4,4), (5,5)]\n    dist_matrix = [[0, 1, 2, 3, 4, 5],\n                   [1, 0, 1, 2, 3, 4],\n                   [2, 1, 0

In [127]:
import random
import math


def swap_cities(best_route):
    # Randomly select two cities in the route and swap their positions
    new_route = best_route.copy()
    city1, city2 = random.sample(range(len(new_route)), 2)
    new_route[city1], new_route[city2] = new_route[city2], new_route[city1]
    return new_route


def two_opt_swap(new_route, dist_matrix):
    # Apply the 2-opt swap to the route to eliminate any crossovers or intersections
    for i in range(len(new_route) - 1):
        for j in range(i + 1, len(new_route)):
            if j - i == 1:
                continue  # changes nothing, skip then
            # reverse the path between i and j
            reversed_path = list(reversed(new_route[i:j]))
            # check if this reverse will cause improvement in distance
            if (dist_matrix[new_route[i - 1]][new_route[j]] + dist_matrix[new_route[i]][new_route[j + 1]]) > (
                    dist_matrix[new_route[i - 1]][new_route[i]] + dist_matrix[new_route[j]][new_route[j + 1]]):
                # apply the reversal
                new_route[i:j] = reversed_path
    return new_route


def convert_distance_unit(best_distance, distance_unit):
    # Convert the best distance to the specified unit of measurement
    if distance_unit == 'km':
        return round(best_distance, 2)
    elif distance_unit == 'm':
        return round(best_distance * 1000, 2)
    else:
        raise ValueError("Invalid distance unit. Choose 'km' or 'm'.")


def distance(coord1, coord2):
    # Calculate the Euclidean distance between two coordinates
    x1, y1 = coord1
    x2, y2 = coord2
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return dist


def generate_initial_route(city_coords):
    # Generate initial route using Randomized Nearest Neighbor algorithm
    city_indices = list(range(len(city_coords)))
    initial_route = [random.choice(city_indices)]
    city_indices.remove(initial_route[0])
    while len(city_indices) > 0:
        last_index = initial_route[-1]
        distances = [(i, distance(city_coords[i], city_coords[last_index]))
                     for i in city_indices]
        distances = sorted(distances, key=lambda x: x[1])
        nearest_index = distances[0][0]
        initial_route.append(nearest_index)
        city_indices.remove(nearest_index)
    return initial_route


def calculate_total_distance(route, dist_matrix):
    # Calculate the total distance of a route given its distance matrix
    total_distance = 0
    for i in range(len(route)-1):
        start_index = route[i]
        end_index = route[i+1]
        total_distance += dist_matrix[start_index][end_index]
    return total_distance


def generate_initial_route(city_coords, dist_matrix):
    # Generate an initial route using the randomized nearest neighbor algorithm
    num_cities = len(city_coords)
    start_city_index = random.randint(0, num_cities-1)
    unvisited_cities = set(range(num_cities))
    unvisited_cities.remove(start_city_index)
    route = [start_city_index]
    while unvisited_cities:
        current_city_index = route[-1]
        nearest_neighbor_index = min(
            unvisited_cities, key=lambda city_index: dist_matrix[current_city_index][city_index])
        route.append(nearest_neighbor_index)
        unvisited_cities.remove(nearest_neighbor_index)
    return route


def run_2opt(route, dist_matrix, num_iterations):
    # Improve a route using the 2-opt algorithm
    best_route = route
    best_distance = calculate_total_distance(best_route, dist_matrix)
    for i in range(num_iterations):
        new_route = best_route.copy()
        # Randomly select two cities to swap
        city1, city2 = random.sample(range(len(new_route)), 2)
        # Swap the two cities to create a new route
        new_route[city1], new_route[city2] = new_route[city2], new_route[city1]
        # Apply 2-opt to the new route
        for j in range(len(new_route)-2):
            for k in range(j+2, len(new_route)-1):
                if dist_matrix[new_route[j]][new_route[k]] + dist_matrix[new_route[j+1]][new_route[k+1]] < dist_matrix[new_route[j]][new_route[j+1]] + dist_matrix[new_route[k]][new_route[k+1]]:
                    new_route[j+1:k+1] = reversed(new_route[j+1:k+1])
        # Calculate the total distance of the new route
        new_distance = calculate_total_distance(new_route, dist_matrix)
        # If the new route is shorter, update the best route and distance
        if new_distance < best_distance:
            best_distance = new_distance
            best_route = new_route
    return best_route, best_distance

def generate_distance_matrix(city_coords):
    dist_matrix = []
    for i, coord1 in enumerate(city_coords):
        row = []
        for j, coord2 in enumerate(city_coords):
            if i == j:
                row.append(0)
            else:
                dist = distance(coord1, coord2)
                row.append(dist)
        dist_matrix.append(row)
    return dist_matrix

def tsp_solver(city_coords, iterations, distance_unit):
    # Generate the distance matrix
    dist_matrix = generate_distance_matrix(city_coords)

    # Generate initial route using Randomized Nearest Neighbor algorithm
    initial_route = generate_initial_route(city_coords, dist_matrix)
    print(f"Initial route: {initial_route}")
    initial_distance = calculate_total_distance(initial_route, dist_matrix)
    print(f"Initial distance: {initial_distance}{distance_unit}")

    # Set best_route to initial route and calculate its total distance
    best_route = initial_route
    best_distance = initial_distance
    print(f"Best route: {best_route}")
    print(f"Best distance: {best_distance}{distance_unit}")

    # Run the 2-opt algorithm for the specified number of iterations
    for i in range(iterations):
        # Randomly select two cities in the route and swap their positions
        new_route = swap_cities(best_route)
        print(f"Iteration {i+1}:")
        print(f"New route: {new_route}")

        # Apply the 2-opt swap to the route to eliminate any crossovers or intersections
        new_route = two_opt_swap(new_route, dist_matrix)
        print(f"2-opt swapped route: {new_route}")

        # Calculate the total distance of the new route
        new_distance = calculate_total_distance(new_route, dist_matrix)
        print(f"New distance: {new_distance}{distance_unit}")

        # If the new route is shorter than the previous best route, update best_route and its total distance
        if new_distance < best_distance:
            best_route = new_route
            best_distance = new_distance
            print(f"New best route: {best_route}")
            print(f"New best distance: {best_distance}{distance_unit}")
        else:
            print(f"No improvement in route for iteration {i+1}")

    # Convert the best distance to the specified unit of measurement
    best_distance = convert_distance_unit(best_distance, distance_unit)

    return best_route, best_distance


city_coords = [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5)]
iterations = 5
distance_unit = 'km'

tsp_solver(city_coords, iterations, distance_unit)

# # TEST CASE 1 - 15 CITIES
# # MAKE CHANGES FOR TEST CASE HERE
# # Set number of iterations and unit of measurement
# num_cities = 100
# num_iterations = 500
# unit = 'km'

# # Generate 15 random cities
# # Author's (Carl's) Note: I wanted to work with the real coordinates of Manila, but I will do that in the future due to time constraints and conflicting priorities.
# city_coords = [(random.uniform(-100, 100), random.uniform(-100, 100))
#                for i in range(num_cities)]

# # Initialize initial route and distance matrix
# dist_matrix = generate_distance_matrix(city_coords)
# initial_route = generate_initial_route(city_coords, dist_matrix)
# # Run RNN + 2-opt algorithm
# best_route, best_distance = run_2opt(
#     initial_route, dist_matrix, num_iterations)
# # # Calculate the percentage difference between initial and best route distances
# initial_distance = calculate_total_distance(initial_route, dist_matrix)
# percent_difference = abs(
#     ((best_distance - initial_distance) / initial_distance) * 100)


# # Print results
# print("++++++++++++++ RESULTS ++++++++++++++")
# print("\nInitial Route:")
# print(initial_route)
# print("Initial Route Distance:", calculate_total_distance(
#     initial_route, dist_matrix), unit)

# print("\nBest Route:")
# print(best_route)
# print("Best Route Distance:", best_distance, unit)

# # Print percentage difference between initial and best route distances
# print("\nThe best_route reduces the distance required to cover by {:.2f}% compared to the intial_route!".format(
#     percent_difference))
# print("++++++++++++++++++++++++++++++++++")


# print("\nCity Coordinates:")
# for city in city_coords:
#     print(city)

# print("\nDistance Matrix:")
# for row in dist_matrix:
#     print(row)


Initial route: [1, 0, 2, 3, 4, 5]
Initial distance: 8.485281374238571km
Best route: [1, 0, 2, 3, 4, 5]
Best distance: 8.485281374238571km
Iteration 1:
New route: [1, 0, 2, 5, 4, 3]


IndexError: list index out of range

In [116]:
# TEST CASE 1 - 15 CITIES
# MAKE CHANGES FOR TEST CASE HERE
# Set number of iterations and unit of measurement
num_cities = 100
num_iterations = 500
unit = 'km'

# Generate 15 random cities
# Author's (Carl's) Note: I wanted to work with the real coordinates of Manila, but I will do that in the future due to time constraints and conflicting priorities.
city_coords = [(random.uniform(-100, 100), random.uniform(-100, 100))
               for i in range(num_cities)]

# Initialize initial route and distance matrix
dist_matrix = generate_distance_matrix(city_coords)
initial_route = generate_initial_route(city_coords, dist_matrix)
# Run RNN + 2-opt algorithm
best_route, best_distance = run_2opt(
    initial_route, dist_matrix, num_iterations)
# # Calculate the percentage difference between initial and best route distances
initial_distance = calculate_total_distance(initial_route, dist_matrix)
percent_difference = abs(
    ((best_distance - initial_distance) / initial_distance) * 100)


# Print results
print("++++++++++++++ RESULTS ++++++++++++++")
print("\nInitial Route:")
print(initial_route)
print("Initial Route Distance:", calculate_total_distance(
    initial_route, dist_matrix), unit)

print("\nBest Route:")
print(best_route)
print("Best Route Distance:", best_distance, unit)

# Print percentage difference between initial and best route distances
print("\nThe best_route reduces the distance required to cover by {:.2f}% compared to the intial_route!".format(
    percent_difference))
print("++++++++++++++++++++++++++++++++++")

print("\nCity Coordinates:")
for city in city_coords:
    print(city)

print("\nDistance Matrix:")
for row in dist_matrix:
    print(row)


++++++++++++++ RESULTS ++++++++++++++

Initial Route:
[19, 66, 51, 18, 71, 28, 13, 93, 14, 69, 76, 25, 85, 74, 38, 65, 88, 32, 35, 33, 73, 39, 20, 78, 26, 81, 60, 36, 27, 6, 29, 98, 75, 70, 3, 11, 2, 40, 16, 89, 46, 9, 91, 5, 86, 22, 10, 55, 34, 59, 12, 63, 42, 37, 95, 45, 15, 30, 67, 90, 56, 23, 57, 58, 92, 68, 44, 48, 21, 0, 62, 4, 47, 82, 52, 84, 24, 31, 94, 61, 17, 80, 79, 83, 77, 99, 87, 54, 50, 96, 1, 41, 97, 64, 49, 43, 7, 53, 8, 72]
Initial Route Distance: 1981.9518203015323 km

Best Route:
[19, 66, 65, 88, 32, 35, 7, 33, 73, 26, 78, 20, 39, 60, 81, 36, 27, 6, 29, 98, 75, 70, 49, 64, 43, 38, 74, 85, 76, 25, 69, 14, 93, 51, 13, 28, 71, 18, 77, 99, 87, 54, 50, 96, 80, 17, 79, 83, 61, 31, 24, 84, 52, 62, 82, 4, 47, 21, 0, 94, 44, 48, 68, 92, 58, 57, 56, 23, 67, 90, 30, 45, 15, 95, 37, 42, 63, 12, 59, 34, 55, 10, 1, 41, 97, 22, 86, 5, 91, 9, 46, 89, 16, 40, 3, 11, 2, 53, 8, 72]
Best Route Distance: 1586.765474462423 km

The best_route reduces the distance required to cover by 19.94

In [23]:
"""
Travelling Salesman Problem Solver

This script implements a heuristic algorithm for the Travelling Salesman Problem (TSP) that uses the 2-opt local search to optimize an initial solution generated by the Randomized Nearest Neighbor algorithm. The TSP seeks to find the shortest route that visits every city in a given list exactly once and returns to the starting city.

This script requires Python 3.6 or later and depends on the `math` module.

Usage:
    Call the `tsp_solver` function with a list of cities and their coordinates, a distance matrix that stores the distance between every pair of cities, the number of iterations to run the algorithm, and the unit of measurement for the distance traveled. The function returns the best route found and its total distance in the specified unit of measurement.

Example:
    cities = [(0,0), (1,1), (2,2), (3,3), (4,4), (5,5)]
    dist_matrix = [[0, 1, 2, 3, 4, 5],
                   [1, 0, 1, 2, 3, 4],
                   [2, 1, 0, 1, 2, 3],
                   [3, 2, 1, 0, 1, 2],
                   [4, 3, 2, 1, 0, 1],
                   [5, 4, 3, 2, 1, 0]]
    iterations = 1000
    unit = 'kilometers'
    best_route, total_distance = tsp_solver(cities, dist_matrix, iterations, unit)
    print(f"Best route: {best_route}\nTotal distance: {total_distance} {unit}")

Functions:
    - `tsp_solver(cities, dist_matrix, iterations, unit)`: Solves the TSP for a given list of cities and their coordinates, distance matrix, number of iterations, and unit of measurement. Returns the best route and its total distance in the specified unit of measurement.

    - `randomized_nearest_neighbor(cities, dist_matrix)`: Generates an initial solution for the TSP using the Randomized Nearest Neighbor algorithm.

    - `two_opt(route, dist_matrix)`: Optimizes a route by applying the 2-opt local search to eliminate crossovers or intersections.

    - `calculate_total_distance(route, dist_matrix)`: Calculates the total distance of a route given its distance matrix.

Author: [Your Name]
Date created: [Date]
Last modified: [Date]
Python Version: 3.6 or later
"""


'\nTravelling Salesman Problem Solver\n\nThis script implements a heuristic algorithm for the Travelling Salesman Problem (TSP) that uses the 2-opt local search to optimize an initial solution generated by the Randomized Nearest Neighbor algorithm. The TSP seeks to find the shortest route that visits every city in a given list exactly once and returns to the starting city.\n\nThis script requires Python 3.6 or later and depends on the `math` module.\n\nUsage:\n    Call the `tsp_solver` function with a list of cities and their coordinates, a distance matrix that stores the distance between every pair of cities, the number of iterations to run the algorithm, and the unit of measurement for the distance traveled. The function returns the best route found and its total distance in the specified unit of measurement.\n\nExample:\n    cities = [(0,0), (1,1), (2,2), (3,3), (4,4), (5,5)]\n    dist_matrix = [[0, 1, 2, 3, 4, 5],\n                   [1, 0, 1, 2, 3, 4],\n                   [2, 1, 0

In [120]:
city_coords = [(0, 0), (3, 4), (1, 1), (2, 2)]
iterations = 10
distance_unit = 'km'

best_route, best_distance = tsp_solver(city_coords, iterations, distance_unit)

print(f"Best route: {best_route}")
print(f"Best distance: {best_distance}{distance_unit}")

TypeError: generate_initial_route() missing 1 required positional argument: 'dist_matrix'

In [133]:
import random
import math


def calculate_distance(city1, city2):
    """
    Calculates the Euclidean distance between two cities.
    """
    return math.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)


def calculate_distance_matrix(cities):
    """
    Generates a distance matrix from a list of cities and their coordinates.
    """
    n = len(cities)
    distance_matrix = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            distance = calculate_distance(cities[i], cities[j])
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance
    return distance_matrix


def randomized_nearest_neighbor(cities, distance_matrix):
    """
    Generates an initial route using the randomized nearest neighbor algorithm.
    """
    n = len(cities)
    visited_cities = [False] * n
    route = [0] * n

    # Select a random starting city.
    current_city = random.randint(0, n-1)
    visited_cities[current_city] = True
    route[0] = current_city

    # Add the remaining cities to the route.
    for i in range(1, n):
        # Compute the distances from the last city added to all remaining cities.
        distances = [(distance_matrix[current_city][j], j)
                     for j in range(n) if not visited_cities[j]]
        # Select the city with the shortest distance to the last city and add it to the route.
        min_distance, nearest_city = min(distances)
        visited_cities[nearest_city] = True
        route[i] = nearest_city
        current_city = nearest_city

    return route


def calculate_total_distance(route, distance_matrix):
    """
    Calculates the total distance of a route given its distance matrix.
    """
    total_distance = 0
    n = len(route)
    for i in range(n-1):
        total_distance += distance_matrix[route[i]][route[i+1]]
    total_distance += distance_matrix[route[-1]][route[0]]
    return total_distance


def swap_cities(route, i, j):
    """
    Swaps two cities in a route.
    """
    new_route = route[:]
    new_route[i], new_route[j] = new_route[j], new_route[i]
    return new_route


def apply_2opt_swap(route, i, j):
    """
    Applies the 2-opt swap to a route.
    """
    new_route = route[:i] + route[i:j+1][::-1] + route[j+1:]
    return new_route


def tsp_algorithm(initial_route, distance_matrix, num_iterations):
    """
    Runs the TSP algorithm for a given number of iterations.
    """
    current_best_route = initial_route
    current_best_distance = calculate_total_distance(
        initial_route, distance_matrix)

    for i in range(num_iterations):
        # Select two random cities in the current best route.
        city1, city2 = random.sample(range(len(initial_route)), 2)
        # Apply the 2-opt swap to the current best route.
        new_route = apply_2opt_swap(current_best_route, city1, city2)

        # Calculate the total distance of the new route.
        new_distance = calculate_total_distance(new_route, distance_matrix)

        # If the new route is shorter than the current best route, update the current best route.
        if new_distance < current_best_distance:
            current_best_route = new_route
            current_best_distance = new_distance
        return current_best_route, current_best_distance


# Define the cities and their coordinates.
cities = [(0, 0), (1, 5), (2, 3), (5, 2), (6, 4), (4, 6), (3, 7),
          (7, 1), (8, 5), (9, 9), (10, 6), (12, 3), (13, 8), (14, 2), (15, 7)]

# Generate the distance matrix and print it.
distance_matrix = calculate_distance_matrix(cities)
print("Distance matrix:")
for row in distance_matrix:
    print(row)

# Generate the initial route using randomized nearest neighbor and print it.
initial_route = randomized_nearest_neighbor(cities, distance_matrix)
print("Initial route (randomized nearest neighbor):", initial_route)


Distance matrix:
[0, 5.0990195135927845, 3.605551275463989, 5.385164807134504, 7.211102550927978, 7.211102550927978, 7.615773105863909, 7.0710678118654755, 9.433981132056603, 12.727922061357855, 11.661903789690601, 12.36931687685298, 15.264337522473747, 14.142135623730951, 16.55294535724685]
[5.0990195135927845, 0, 2.23606797749979, 5.0, 5.0990195135927845, 3.1622776601683795, 2.8284271247461903, 7.211102550927978, 7.0, 8.94427190999916, 9.055385138137417, 11.180339887498949, 12.36931687685298, 13.341664064126334, 14.142135623730951]
[3.605551275463989, 2.23606797749979, 0, 3.1622776601683795, 4.123105625617661, 3.605551275463989, 4.123105625617661, 5.385164807134504, 6.324555320336759, 9.219544457292887, 8.54400374531753, 10.0, 12.083045973594572, 12.041594578792296, 13.601470508735444]
[5.385164807134504, 5.0, 3.1622776601683795, 0, 2.23606797749979, 4.123105625617661, 5.385164807134504, 2.23606797749979, 4.242640687119285, 8.06225774829855, 6.4031242374328485, 7.0710678118654755, 10

In [129]:
import time

# Generate a list of 15 cities with random coordinates.
cities1 = [(random.uniform(0, 100), random.uniform(0, 100)) for i in range(15)]
# Generate a distance matrix from the list of cities.
distance_matrix1 = calculate_distance_matrix(cities1)

# Generate an initial route using the randomized nearest neighbor algorithm.
initial_route1 = randomized_nearest_neighbor(cities1, distance_matrix1)

# Run the TSP algorithm for 500 iterations and measure the execution time.
start_time = time.time()
final_route1, final_distance1 = tsp_algorithm(
    initial_route1, distance_matrix1, 500)
end_time = time.time()

print("Test case 1: n=15 cities and 500 iterations")
print("Initial route: {}".format(initial_route1))
print("Initial distance: {}".format(
    calculate_total_distance(initial_route1, distance_matrix1)))
print("Final route: {}".format(final_route1))
print("Final distance: {}".format(final_distance1))
print("Execution time: {} seconds".format(end_time - start_time))


Test case 1: n=15 cities and 500 iterations
Initial route: [13, 5, 14, 4, 0, 11, 2, 3, 12, 1, 9, 7, 10, 6, 8]
Initial distance: 308.8144636039584
Final route: [13, 5, 14, 4, 0, 11, 2, 3, 12, 1, 9, 7, 10, 6, 8]
Final distance: 308.8144636039584
Execution time: 0.011114358901977539 seconds


In [130]:
# Generate a list of 50 cities with random coordinates.
cities2 = [(random.uniform(0, 100), random.uniform(0, 100)) for i in range(50)]
# Generate a distance matrix from the list of cities.
distance_matrix2 = calculate_distance_matrix(cities2)

# Generate an initial route using the randomized nearest neighbor algorithm.
initial_route2 = randomized_nearest_neighbor(cities2, distance_matrix2)

# Run the TSP algorithm for 1000 iterations and measure the execution time.
start_time = time.time()
final_route2, final_distance2 = tsp_algorithm(
    initial_route2, distance_matrix2, 1000)
end_time = time.time()

print("Test case 2: n=50 cities and 1000 iterations")
print("Initial route: {}".format(initial_route2))
print("Initial distance: {}".format(
    calculate_total_distance(initial_route2, distance_matrix2)))
print("Final route: {}".format(final_route2))
print("Final distance: {}".format(final_distance2))
print("Execution time: {} seconds".format(end_time - start_time))


Test case 2: n=50 cities and 1000 iterations
Initial route: [11, 48, 30, 34, 22, 1, 17, 18, 38, 0, 4, 5, 15, 6, 12, 44, 35, 8, 13, 47, 26, 16, 43, 10, 9, 7, 14, 28, 2, 36, 31, 49, 42, 40, 29, 32, 3, 24, 21, 46, 33, 25, 45, 19, 23, 20, 37, 27, 39, 41]
Initial distance: 668.4677077695392
Final route: [11, 48, 30, 34, 22, 1, 17, 18, 38, 0, 4, 5, 35, 44, 12, 6, 15, 8, 13, 47, 26, 16, 43, 10, 9, 7, 14, 28, 2, 36, 31, 49, 42, 29, 40, 32, 3, 24, 21, 46, 33, 25, 45, 19, 41, 39, 20, 37, 27, 23]
Final distance: 621.636509454927
Execution time: 0.015583038330078125 seconds
